In [67]:
import requests
import sys, os
from dotenv import load_dotenv
current_dir = os.getcwd()
parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)
from security_routes.utilities import hash_password
from pathlib import Path

In [68]:
#base_url = "https://tmv9bz5v4q.us-east-1.awsapprunner.com/"
base_url = "http://localhost:8000/"
prefix = "latest"

In [69]:
hash_password("password")

'$2b$12$dZI5kdoVzBo/gMbHXLQNLew33BrYs0CyqKquk2BJk.LfxhFs2lR9O'

In [70]:
load_dotenv("../.env", override=True)

True

In [71]:
adminpassword = os.getenv("ADMIN_PASSWORD")  
adminpassword is not None

True

In [73]:
response = requests.post(
    base_url+"latest/token", data={"username": "admin", "password": "adminpassword"}
)

admin_token = response.json()["access_token"]

In [74]:
len(admin_token)

145

Create user and group

In [75]:
new_user_data = {
    "username": "new_user",
    "email": "new_user@example.com",
    "password": "newpassword",
    "is_admin": False,
}

response = requests.post(
    base_url+f"{prefix}/users",
    params=new_user_data,
    headers={"Authorization": f"Bearer {admin_token}"},
)

response

<Response [200]>

In [76]:
group_data = {
    "name": "new_group",
    "description": "A new group",
}
response = requests.post(
    base_url+f"{prefix}/groups",
    params=group_data,
    headers={"Authorization": f"Bearer {admin_token}"},
)
response

<Response [200]>

In [77]:
link_data = {
    "username": "new_user",
    "groupname": "new_group",
}
response = requests.post(
    base_url+f"{prefix}/link-user-group",
    params=link_data,
    headers={"Authorization": f"Bearer {admin_token}"},
)
response

<Response [201]>

Create a revision

In [78]:
response = requests.post(
    base_url+"latest/token", data={"username": "new_user", "password": "newpassword"}
)
token = response.json()["access_token"]

In [79]:
len(token)

151

In [88]:
new_version_data = {
    "name": "New Version",
    "iso_language": "eng",
    "iso_script": "Latn",
    "abbreviation": "NV",
    "rights": "Some Rights",
    "machineTranslation": False,
}

response = requests.post(
    base_url+f"{prefix}/version",
    params=new_version_data,
    headers={"Authorization": f"Bearer {token}"},
)
version_id =response.json()["id"]
response.json()

KeyError: 'id'

In [89]:
print(new_version_data)

{'name': 'New Version', 'iso_language': 'eng', 'iso_script': 'Latn', 'abbreviation': 'NV', 'rights': 'Some Rights', 'machineTranslation': 'False'}


In [90]:
response.text

'{"detail":[{"type":"missing","loc":["body"],"msg":"Field required","input":null,"url":"https://errors.pydantic.dev/2.4/v/missing"}]}'

In [15]:
headers = {"Authorization": f"Bearer {token}"}
test_revision = {
    "version_id": version_id,
    "name": "Test Revision",
}
test_upload_file = Path("../fixtures/uploadtest.txt")

with open(test_upload_file, "rb") as file:
    files = {"file": file}
    response = requests.post(
    base_url + f"{prefix}/revision", params=test_revision, files=files, headers=headers
    )
revision_id =response.json()["id"]
response.json()

{'id': 1,
 'bible_version_id': 2,
 'version_abbreviation': 'NV',
 'date': '2024-02-13',
 'name': 'Test Revision',
 'published': False,
 'backTranslation': None,
 'machineTranslation': False,
 'iso_language': 'eng'}

In [16]:
response = requests.get(
    base_url+f"{prefix}/revision",
    headers={"Authorization": f"Bearer {token}"},
)
response.json()

[{'id': 1,
  'bible_version_id': 2,
  'version_abbreviation': 'NV',
  'date': '2024-02-13',
  'name': 'Test Revision',
  'published': False,
  'backTranslation': None,
  'machineTranslation': False,
  'iso_language': 'eng'}]

In [17]:
response = requests.get(
    base_url+f"{prefix}/version",
    headers={"Authorization": f"Bearer {token}"},
)
response.json()

[{'id': 2,
  'name': 'New Version',
  'iso_language': 'eng',
  'iso_script': 'Latn',
  'abbreviation': 'NV',
  'rights': 'Some Rights',
  'forwardTranslation': None,
  'backTranslation': None,
  'machineTranslation': False}]

Delete version and revision

In [18]:
response = requests.delete(
    base_url+f"{prefix}/revision",
    params={"id": revision_id},
    headers={"Authorization": f"Bearer {token}"},
)
response.json()

{'detail': 'Revision 1 deleted successfully.'}

In [19]:
response = requests.delete(
    base_url+f"{prefix}/version",
    params={"id": version_id},
    headers={"Authorization": f"Bearer {token}"},
)
response.json()

{'detail': 'Version New Version successfully deleted.'}

Delete user and group

In [20]:
response = requests.delete(
    base_url+f"{prefix}/users",
    params={"username": "new_user"},
    headers={"Authorization": f"Bearer {admin_token}"},
)
response

<Response [204]>

In [21]:
response = requests.delete(
    base_url+f"{prefix}/groups",
    params={"groupname": "new_group"},
    headers={"Authorization": f"Bearer {admin_token}"},
)
response

<Response [204]>

In [ ]:
headers = {"Authorization": f"Bearer {token}"}
response = requests.get(f"{base_url}/v3/language", headers=headers)
assert response.status_code == 200
assert response.json() == [{"iso639": "eng", "name": "english"}]